## Set-up
### Dependencies
We use `scikit-learn` for implementing models and `sklearn-rvm` for a Relevance Vector Machine (RVM) implementation. We also use `numpy` and `pandas` for processing data into nice data frames.
### Constants

In [1]:
import numpy as np

SEED = 24
BIN_COUNT = 50
CV_COUNT = 10

np.random.seed(SEED)

### Utility Functions

In [87]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
import json


def get_bin_values_from_sketch(fname: str) -> list[int]:
    try:
        json_data = open(f"../sketches/sample_{fname}.json")
        return json.load(json_data)["signatures"][0]["Sketch"]["mins"]
    except:
        return [-1] * BIN_COUNT


def get_train_test_f1_score(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    return f1_score(y_train, clf.predict(X_train)), f1_score(y_test, clf.predict(X_test))


def get_f1_results(clf, X, y):
    metrics = ["Train", "Test", "Mean CV", "Std CV"]

    if len(X) == 0:
        return dict(zip(metrics, ["n/a"] * len(metrics)))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2)
    f1_train, f1_test = get_train_test_f1_score(clf,
                                                X_train, y_train, X_test, y_test)
    X_train, y_train = shuffle(X_train, y_train)
    cv_score = cross_val_score(
        clf, X_train, y_train, scoring="f1", cv=CV_COUNT)
    return dict(zip(metrics, (f1_train, f1_test, cv_score.mean(), cv_score.std())))


### Loading Data
The subject metadata of the infant study can be found on the [T1D cohort WGS page on Diabimmune](https://diabimmune.broadinstitute.org/diabimmune/t1d-cohort/resources/metagenomic-sequence-data). The gid and diagnosis are used to retrieve the relevant sketches and test predictive accuracy respectively.

In [88]:
import pandas as pd

# Metadata on the study
df = pd.read_csv("../data/diabimmune_t1d_wgs_metadata.csv")[
    ["Gid_shotgun", "T1D_Diagnosed"]]

# Histosketch bin values
X = np.stack(df["Gid_shotgun"].apply(
    get_bin_values_from_sketch).values)
features = np.arange(BIN_COUNT)

# T1D diagnosis
y = df["T1D_Diagnosed"].apply(int).to_numpy()[X[:,0] > 0]

# Remove negative frequencies from invalid sketch files
X = X[X[:,0] > 0]


### Feature Engineering
We use lasso to isolate statistically relevant features. Both the entire bin count and ones chosen by lasso are used.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

pipeline = make_pipeline(StandardScaler(), Lasso())

# Find best alpha parameter for lasso via 10-fold CV
search = GridSearchCV(pipeline, {"lasso__alpha": np.arange(
    0.1, 10, 0.1)}, cv=CV_COUNT, scoring="neg_mean_squared_error")
search.fit(X_train, y_train)
coefficients = search.best_estimator_.named_steps["lasso"].coef_

# Retrieve features with significance
lasso_features = features[np.abs(coefficients) > 0]

feature_map = {"Regular": features, "Lasso": lasso_features}
feature_map

## Testing
### Model Fitting
The F1 scores are calculated for each model. Models used are: Relevance Vector Machines (RVM), Support Vector Machines (SVM), Random Forests (RF), and Naive Bayes (NB).

In [85]:
from sklearn_rvm import EMRVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

classifiers = {
    "RVM": EMRVC(gamma="scale", kernel="rbf"),
    "SVM": SVC(gamma="scale", kernel="rbf"),
    "RF": RandomForestClassifier(),
    "NB": GaussianNB(),
}


### Results
Metrics for performance are identical to the original study: finding the F1 scores for training, testing, and 10-fold CV of each model. These are done twice: with all bins as features as well as only those chosen by lasso.

In [ ]:
results = pd.DataFrame(dict((f"{name} {suffix}", get_f1_results(clf, X[:,features_chosen], y)) for (
    name, clf) in classifiers.items() for (suffix, features_chosen) in feature_map.items()))

results